In [1]:
from datetime import datetime
from typing import List, Sequence
from config import OPENAI_API_KEY, TAVILY_API_KEY, LANGCHAIN_TRACING_V2, LANGCHAIN_API_KEY
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser, PydanticToolsParser
from langgraph.graph import END, MessageGraph
from langchain_openai import ChatOpenAI
from schemas import AnswerQuestion

In [2]:
llm = ChatOpenAI(model="gpt-4-turbo-2024-04-09")
# it takes the output from the llm and transform it into a dictionary
parser = JsonOutputToolsParser(return_id=True)
# it takes the output from the llm and transform it into an AnswerQuestion object
parser_pydantic =  PydanticToolsParser(tools=[AnswerQuestion])

In [3]:
actor_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system",
        """
        You are an expert researcher.
        Current time: {time}
        
        1. {first_instruction}
        2. Reflect and Critique your answer. Be severe to maximize improvements.
        3. Recommend search queries to research information and improve your answer.""",
        ),
        MessagesPlaceholder(variable_name="messages"),
        
    ]
).partial(time=lambda: datetime.now().isoformat())

In [4]:
first_responder_prompt_template = actor_prompt_template.partial(first_instruction="Provide a detailed ~500 word answer.")

In [5]:
first_responder = first_responder_prompt_template | llm.bind_tools(tools=[AnswerQuestion], tool_choice="AnswerQuestion")

In [6]:
human_message = HumanMessage(
    content="Write an article on the latest trend of Generative AI")

In [7]:
chain =  first_responder_prompt_template |  llm.bind_tools(tools=[AnswerQuestion], tool_choice="AnswerQuestion") | parser_pydantic

In [8]:
res = chain.invoke(input={"messages": [human_message]})

In [9]:
print(res)

[AnswerQuestion(answer="Generative AI has recently been at the forefront of technological advancements, sparking significant interest across various sectors. This latest trend in artificial intelligence involves systems that can generate text, images, music, and other forms of media, essentially creating new content based on the data they have been trained on. The core technology behind this is often rooted in machine learning models like GANs (Generative Adversarial Networks) and transformers, which have shown remarkable ability in understanding and generating human-like content.\n\nOne of the most prominent examples of generative AI is OpenAI's ChatGPT, which has gained widespread attention for its ability to generate coherent and contextually appropriate text based on user prompts. This technology has not only transformed consumer applications but also has profound implications for businesses, enhancing automation and personalization in various industries including marketing, journa

In [10]:
res[0].answer

"Generative AI has recently been at the forefront of technological advancements, sparking significant interest across various sectors. This latest trend in artificial intelligence involves systems that can generate text, images, music, and other forms of media, essentially creating new content based on the data they have been trained on. The core technology behind this is often rooted in machine learning models like GANs (Generative Adversarial Networks) and transformers, which have shown remarkable ability in understanding and generating human-like content.\n\nOne of the most prominent examples of generative AI is OpenAI's ChatGPT, which has gained widespread attention for its ability to generate coherent and contextually appropriate text based on user prompts. This technology has not only transformed consumer applications but also has profound implications for businesses, enhancing automation and personalization in various industries including marketing, journalism, and entertainment

In [11]:
res[0].reflection

Reflection(missing='While the answer provides a broad overview of generative AI, it lacks specific recent developments or examples of cutting-edge applications. It could also benefit from more detailed discussion on the technical advancements and the specific AI models being used.', superfluous='The answer might be overly broad and generic, lacking depth in any particular aspect which could engage readers more effectively. Some of the general statements about the impact of generative AI could be considered as filler content.')

In [12]:
res[0].search_queries

['latest advancements in generative AI 2024',
 'specific applications of generative AI in industries',
 'technical details of AI models used in generative AI']